# Make S2 Geomedian with different configurations

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pyproj import Transformer

import datacube

import sys
sys.path.append('../Scripts')
from deafrica_dask import create_local_dask_cluster
from s2_geomedian import run_tile


In [ ]:
create_local_dask_cluster(n_workers=14,threads_per_worker=1)

## Generate seasonal geomedian for 2017 and 2019

In [ ]:

def run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=False, fourmonthly=False, tmad=False, outpath='s2_geomedian'):
    
    # convert to projected centre coordinates
    transformer = Transformer.from_crs("epsg:4326", "epsg:6933", always_xy=True)
    x0, y0 = transformer.transform(lon, lat)

    x = (x0-tilesize/2, x0+tilesize/2)
    y = (y0-tilesize/2, y0+tilesize/2)

    if annual_only:
        times = [("2017-01-01", "2017-12-31"), ("2019-01-01", "2019-12-31")]
    elif fourmonthly:
        times = [
            ("2019-01-01", "2019-04-30"),
            ("2019-05-01", "2019-08-31"),
            ("2019-09-01", "2019-12-31"),]
    else:
        times = [
        # quarter
        ("2017-01-01", "2017-03-31"),
        ("2017-04-01", "2017-06-30"),
        ("2017-07-01", "2017-09-30"),
        ("2017-10-01", "2017-12-31"),
        # 6 month
        #("2017-01-01", "2017-06-30"),
        #("2017-07-01", "2017-12-31"),
        # 12 month
        ("2017-01-01", "2017-12-31"),
        # quarter
        ("2019-01-01", "2019-03-31"),
        ("2019-04-01", "2019-06-30"),
        ("2019-07-01", "2019-09-30"),
        ("2019-10-01", "2019-12-31"),
        # 6 month
        #("2019-01-01", "2019-06-30"),
        #("2019-07-01", "2019-12-31"),
        # 12 month
        ("2019-01-01", "2019-12-31"),
        ]

    dask_chunks = {'time': -1, 'x':200, 'y': 200}

    filenames = []
    for time in times:
        if cloud_buffer>0:
            filename = "%s/cloud_cover_0_%d/s2_%s_%s_%d_%d_buffer%d.nc"%(outpath, maxcloud, time[0], time[1], x0, y0, cloud_buffer)
        else:
            filename = "%s/cloud_cover_0_%d/s2_%s_%s_%d_%d.nc"%(outpath, maxcloud, time[0], time[1], x0, y0)
        print(filename)
        filenames.append(filename)
    
        gm = run_tile(x, y, time, filename,
                  cloud_labels=[8,9,10], cloud_buffer=cloud_buffer, bad_labels=[0,1,3], 
                  bands=['blue','green','red','nir_2','swir_1'], # only testing a few bands for speed
                  redo=False, tmad=tmad,
                  dask_chunks = dask_chunks, cloud_cover=[0, maxcloud],
                 )


## Cloud cover filtering

In [ ]:
maxcloud = 60

print('Buruni ==============================================================================')
lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
#cloud_buffer=10
#run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

print('Lagos, Nigeria ==============================================================================')
lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

lat, lon = 6.6, 3.3
tilesize = 50000
#cloud_buffer=10
#run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

print('Congo/Gabon ==============================================================================')
lat, lon = -2.3, 12.8
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

#cloud_buffer=10
#run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

print('South Africa ==============================================================================')
lat, lon = -34.2, 19.7
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

print('North Angola ==============================================================================')
lat, lon = -7.7, 13.2
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)


print('Tanzania ==============================================================================')
lat, lon = -9.2, 34.9
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)


print('Cairo ==============================================================================')
lat, lon = 30, 31.17
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)


print('salt pan, Botswana ==============================================================================')
lat, lon = -20.7, 25.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)



In [ ]:
maxcloud = 100

print('Buruni ==============================================================================')
lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Lagos, Nigeria ==============================================================================')
lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('South Africa ==============================================================================')
lat, lon = -34.2, 19.7
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Cairo ==============================================================================')
lat, lon = 30, 31.17
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('North Angola ==============================================================================')
lat, lon = -7.7, 13.2
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Tanzania ==============================================================================')
lat, lon = -9.2, 34.9
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('salt pan, Botswana ==============================================================================')
lat, lon = -20.7, 25.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Congo/Gabon ==============================================================================')
lat, lon = -2.3, 12.8
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud)

#cloud_buffer=10
#run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)


In [ ]:
maxcloud = 40

print('North Angola ==============================================================================')
lat, lon = -7.7, 13.2
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)


print('Tanzania ==============================================================================')
lat, lon = -9.2, 34.9
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)


print('Buruni ==============================================================================')
lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)


print('Lagos, Nigeria ==============================================================================')
lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)


print('salt pan, Botswana ==============================================================================')
lat, lon = -20.7, 25.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Congo/Gabon ==============================================================================')
lat, lon = -2.3, 12.8
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('South Africa ==============================================================================')
lat, lon = -34.2, 19.7
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)

print('Cairo ==============================================================================')
lat, lon = 30, 31.17
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True)
